# Extract paragraphs, combine short paragraphs

In [119]:
import re
import os
import csv
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
import pandas as pd
import numpy as np
import random


[nltk_data] Downloading package punkt to /Users/yuchenluo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# New Loop

In [38]:
col_names = ["text","candidate", "term", "title", "comp"]
rows = []
for root, dirs, files, in os.walk('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse'):
    for file in files:
        if file.endswith('txt'):
            speech = open(os.path.join(root, file), "r").read()
            #speech = f.read()
            #get meta data of the speech
            cand = re.sub("-speech.*", "", file)
            term = re.search("speech-(.*)-", file).group(1)[0:4]
            title = re.search("speech-(.*).txt", file).group(1)
            pars = speech.split('\n')
            pars = [i for i in pars if i] #remove empty strings
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets
            # loop through all paragraphs inside each speech 
            i = 0    
            while i < len(pars):
                n_sent = len(sent_tokenize(pars[i]))
                if n_sent >2:
                    row = [pars[i], cand, term, title, False]
                    rows.append(row)
                    i += 1
                elif i < len(pars) - 1:
                    row = [pars[i] + " " + pars[i +1], cand, term, title, True]
                    i += 2
                    if len(sent_tokenize(row[0])) < 3 and i < len(pars):
                        row_2 = [row[0] + " " + pars[i], cand, term, title, True]
                        i += 1
                        rows.append(row_2)
                    else: 
                        rows.append(row)
                elif i == (len(pars) - 1): 
                    row = [pars[i], cand, term, title, False]
                    rows.append(row)
                    i += 1    

In [39]:
rows

[['We meet here at a time of great uncertainty for America. The era of greed and irresponsibility on Wall Street and in Washington has led us to a financial crisis as serious as any we have faced since the Great Depression. They said they wanted to let the market run free but they let it run wild, and they trampled our American values of fairness, balance, and responsibility to one another. Now, because of speculators who gamed the system and regulators who looked the other way, your jobs, your life savings, and the stability of our entire economy are at risk.',
  'obama',
  '2008',
  '2008-09-29-remarks-westminster-colorado',
  False],
 ["We have been left with no good options. And today, Democrats and Republicans in Washington have agreed on an emergency rescue plan that is our best and only way to prevent an economic catastrophe. It's a plan that's improved a good deal over the last week. This Administration started off by asking for a blank check to solve this problem. I said absol

In [40]:
with open("paragraphs.csv", 'w') as csvfile:
    writer = csv.writer(csvfile)   
    writer.writerow(col_names)
    writer.writerows(rows)

# Old Loop

In [41]:
col_names = ["text","candidate", "term", "title", "comp"]
rows = []
for root, dirs, files, in os.walk('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse'):
    for file in files:
        if file.endswith('txt'):
            speech = open(os.path.join(root, file), "r").read()
            #speech = f.read()
            #get meta data of the speech
            cand = re.sub("-speech.*", "", file)
            term = re.search("speech-(.*)-", file).group(1)[0:4]
            title = re.search("speech-(.*).txt", file).group(1)
            pars = speech.split('\n')
            pars = [i for i in pars if i] #remove empty strings
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets
            i = 0     
            while i < (len(pars) - 1):
                n_sent = len(sent_tokenize(pars[i]))
                if n_sent >2:
                    row = [pars[i], cand, term, title, False]
                    rows.append(row)
                    i += 1
                elif i < len(pars) - 1:
                    row = [pars[i] + " " + pars[i +1], cand, term, title, True]
                    i += 2
                    if len(sent_tokenize(row[0])) < 3 and i < len(pars) - 1:
                        row_2 = [row[0] + " " + pars[i], cand, term, title, True]
                        i += 1
                        rows.append(row_2)
                    else: 
                        rows.append(row)
                else: 
                    row = [pars[i - 1] + " " + pars[i], cand, term, title, True]
                    print(i)
                    rows.pop()
                    rows.append(row)

In [42]:
col_names = ["text","candidate", "term", "title", "comp"]
with open("paragraphs_old.csv", 'w') as csvfile:
    writer = csv.writer(csvfile)   
    writer.writerow(col_names)
    writer.writerows(rows)

In [43]:
paragraphs_new = pd.read_csv("paragraphs.csv")
paragraphs_old = pd.read_csv("paragraphs_old.csv")

In [59]:
# compare the old and the new one
df = pd.merge(paragraphs_old, paragraphs_new, on="text", how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)


In [67]:
print(len(paragraphs_new))
print(len(paragraphs_old))
df.to_csv("paragraphs_dropped.csv")

# Repeat the process for Annenberg data

In [15]:
import pandas as pd
annen_meta = pd.read_csv("/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/annenberg_metadata.csv")

In [29]:
root = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data" 
col_names = ["Speech_id","text","party", "term", "comp"] 
rows = [] 

for file in os.listdir('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data'): 
    if file.endswith('txt'): 
        speech = open(os.path.join(root, file), "r").read() 
        #get meta data of the speech 
        id_speech = re.search('File_(.*).txt', file).group(1) 
        party = annen_meta[annen_meta['id_speech'] == float(id_speech)]['party'].iloc[0] 
        term = annen_meta[annen_meta['id_speech'] == float(id_speech)]['year'].iloc[0] 
        pars = speech.split('\n\n') 
        Type = annen_meta[annen_meta['id_speech'] == float(id_speech)]['type'].iloc[0] 
        if Type == 'Speeches': #only keep speeches 
        # the last paragraph is not part of body text
            pars = [i for i in pars if i] #remove empty strings 
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets 
            pars = [re.sub("\t", "",par) for par in pars]
            pars = [re.sub("\n", "",par) for par in pars] # remove \t and \n
            # loop through all paragraphs inside each speech  
            i = 0    
            while i < len(pars):
                n_sent = len(sent_tokenize(pars[i]))
                if n_sent >2:
                    row = [id_speech, pars[i], party, term, False]
                    rows.append(row)
                    i += 1
                elif i < len(pars) - 1:
                    row = [id_speech, pars[i] + " " + pars[i +1], party, term, True]
                    i += 2
                    if len(sent_tokenize(row[1])) < 3 and i < len(pars):
                        row_2 = [id_speech, row[1] + " " + pars[i], party, term,True]
                        i += 1
                        rows.append(row_2)
                    else: 
                        rows.append(row)
                elif i == (len(pars) - 1): 
                    row = [id_speech,rows[-1][1] + " " + pars[i], party, term, True]
                    rows.pop()
                    rows.append(row)
                    i += 1  

In [30]:
with open("paragraphs_annenberg.csv", 'w') as csvfile: 
    writer = csv.writer(csvfile)    
    writer.writerow(col_names) 
    writer.writerows(rows)

In [40]:
rows[-1][1]

'Be of good cheer. And remember, my dear friends, what a wise man said, "A merry heart doeth good like a medicine, but a broken spirit dryeth the bones." DEMOCRACYELECTIONS--CONCESSION ADDRESSES\x0f\x00\x00\x00\x00\x00\x07\x0e\x01\x00\x02\x00\x03\x00\x04\x00\x00\x0c\x00'

# Old Loop

In [32]:
root = "/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data" 
col_names = ["Speech_id","text","party", "term", "comp"] 
rows = [] 
for file in os.listdir('/Users/yuchenluo/Desktop/Measure_RadicalDiscourse/Annenberg-data'): 
    if file.endswith('txt'): 
        speech = open(os.path.join(root, file), "r").read() 
        #get meta data of the speech 
        id_speech = re.search('File_(.*).txt', file).group(1) 
        party = annen_meta[annen_meta['id_speech'] == float(id_speech)]['party'].iloc[0] 
        term = annen_meta[annen_meta['id_speech'] == float(id_speech)]['year'].iloc[0] 
        pars = speech.split('\n\n') 
        Type = annen_meta[annen_meta['id_speech'] == float(id_speech)]['type'].iloc[0] 
        if Type == 'Speeches': #only keep speeches 
        # the last paragraph is not part of body text
            del pars[-1] #delete last paragraph
            pars = [i for i in pars if i] #remove empty strings 
            pars = [re.sub(" \[[^()]*\]", "", par) for par in pars] #remove strings inside brackets 
            pars = [re.sub("\t", "",par) for par in pars]
            pars = [re.sub("\n", "",par) for par in pars] # remove \t and \n
            # loop through all paragraphs inside each speech  
            i = 0     
            while i < (len(pars) - 1):
                n_sent = len(sent_tokenize(pars[i]))
                if n_sent >2:
                    row = [id_speech,pars[i], party, term, False]
                    rows.append(row)
                    i += 1
                elif i < len(pars) - 1:
                    row = [id_speech,pars[i] + " " + pars[i +1], party, term, True]
                    i += 2
                    if len(sent_tokenize(row[1])) < 3 and i < len(pars) - 1:
                        row_2 = [id_speech,row[1] + " " + pars[i], party, term, True]
                        i += 1
                        rows.append(row_2)
                    else: 
                        rows.append(row)
                else: 
                    row = [id_speech,pars[i - 1] + " " + pars[i], party, term, True]
                    print(i)
                    rows.pop()
                    rows.append(row)

In [33]:
with open("paragraphs_annenberg_old.csv", 'w') as csvfile: 
    writer = csv.writer(csvfile)    
    writer.writerow(col_names) 
    writer.writerows(rows)

In [34]:
paragraphs_new = pd.read_csv("paragraphs_annenberg.csv")
paragraphs_old = pd.read_csv("paragraphs_annenberg_old.csv")
# compare the old and the new one
df = pd.merge(paragraphs_old, paragraphs_new, on="text", how='outer', indicator=True).query("_merge != 'both'").drop('_merge', axis=1).reset_index(drop=True)

In [35]:
df

,Speech_id_x,text,party_x,term_x,comp_x,Speech_id_y,party_y,term_y,comp_y
0,2696.0,Only you can put peace and prosperity within o...,rep,1980.0,True,NaN,NaN,NaN,NaN
1,2899.0,But for those of us who have seen and talked t...,dem,1988.0,True,NaN,NaN,NaN,NaN
2,1406.0,I can't tell you how happy I am that there is ...,rep,1956.0,True,NaN,NaN,NaN,NaN
3,2457.0,And let me conclude by saying: Jerry Ford hasn...,rep,1976.0,True,NaN,NaN,NaN,NaN
4,2286.0,So let us unite to cast off the doubts and dec...,dem,1972.0,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
3584,NaN,ALLIES/ALLIANCESSOVIET UNION (COLD WAR ERA)MID...,NaN,NaN,NaN,1386.0,dem,1956.0,False
3585,NaN,And now I'll just finish by saying something t...,NaN,NaN,NaN,2857.0,rep,1984.0,True
3586,NaN,ELECTIONS--OPPONENT'S CAMPAIGNPARTY UNITY/LOYA...,NaN,NaN,NaN,923.0,dem,1952.0,False
3587,NaN,"CONDUCT, OFFICIAL--CIVILITY",NaN,NaN,NaN,937.0,dem,1952.0,False


In [37]:
df.to_csv("paragraphs_dropped_annenberg.csv")

In [38]:
len(rows)

51201

# Combine UCSB data and Annenberg data


In [43]:
import numpy as np

ucsb = pd.read_csv('paragraphs.csv')
annenberg = pd.read_csv('paragraphs_annenberg.csv',skipinitialspace=True)

# remove \n and \t in annenberg data 
annenberg['text'] = annenberg['text'].replace(r'\n',' ', regex=True) 
annenberg['text'] = annenberg['text'].replace(r'\t',' ', regex=True) 

In [44]:
# create a party variable for ucsb data to match the annenberg data
dems = ['obama', 'gore-jr', 'clinton','kerry']
ucsb['party'] = ['dem' if x in dems else 'rep' for x in ucsb['candidate']] 

# match columns
ucsb_matched = ucsb[['title', 'text', 'party', 'term', 'comp']]
ucsb_matched.rename(columns = {'title':'Speech_id'}, inplace = True) 


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [45]:
all_data = annenberg.append(ucsb_matched)

In [142]:
# add paragraph id
all_dt = pd.DataFrame(columns = ['Speech_id', 'text', 'party', 'term', 'comp', 'par_id'])
current_id = all_data['Speech_id'].iloc[0]
par_id = 0
for index, row in all_data.iterrows():
    speech_id = row['Speech_id']
    par_id += 1
    if speech_id != current_id:
        current_id = speech_id
        par_id = 0
    new_row = row
    new_row['par_id'] = par_id
    all_dt = all_dt.append(new_row)
#     print(new_row)
#     speech = all_data[all_data['Speech_id'] == sp_id]
#     for i in range(len(speech)):
#         par = speech.loc[i]
#         par['par_id'] = i + 1
#         #all_dt.append(par)
#         print(par)
#     for ind in speech.index: 
#         i = 1
#         speech['par_id'][ind] = i
#         i += 1
#         print(speech)
#     for index, row in speech.iterrows(): 
#         i = 1
#         row['par_id'] = i
#         i += 1
#         print(row)

In [160]:
all_data.to_csv('all_paragraphs.csv')

# Sample the data

In [163]:
import random
random.seed(11)
sample = all_data.sample(n = 1000)
sample1 = sample[0:500]
sample2 = sample[500:]

In [162]:
sample1.to_csv('yuchen_sample.csv')
sample2.to_csv('bart_sample.csv')

In [145]:
all_dt.to_csv('NEW_all_paragraphs.csv')

In [156]:
all_data.index = range(0,len(all_data))
all_data

In [3]:
all_dt = pd.read_csv('20201115_all_paragraphs.csv')
#print(len(all_dt))
sample1 = pd.read_csv('yuchen_sample.csv')
sample2 = pd.read_csv('bart_sample.csv')
sample3 = pd.read_csv('yuchen_2nd_sample.csv')
sample4 = pd.read_csv('bart_2nd_sample.csv')
sample5 = pd.read_csv('oscar_sample.csv')

sample = sample1.append(sample2.append(sample3.append(sample4.append(sample5,ignore_index=True))))
sample['total_id'] = sample['Speech_id'].astype(str)  + '_' + sample['par_id'].astype(str) 
# all_dt['total_id'] = all_dt['Speech_id'].astype(str)  + '_' + all_dt['par_id'].astype(str) 

sample

,Unnamed: 0,Speech_id,text,party,term,comp,Unnamed: 0.1,populist_old_keywords,par_id,total_id
0,15777,2192,The fact of the matter is that we find in the ...,rep,1968,True,NaN,NaN,NaN,2192_nan
1,8283,1987,The first thing you do Tuesday morning is get ...,dem,1964,False,NaN,NaN,NaN,1987_nan
2,50267,3156,"Governor Clinton is talking about ""Well, we re...",rep,1992,False,NaN,NaN,NaN,3156_nan
3,12341,1402,"In the meantime, wages have been raised and th...",rep,1956,False,NaN,NaN,NaN,1402_nan
4,44383,3226,"And when people need it, they're smart enough ...",dem,1996,False,NaN,NaN,NaN,3226_nan
...,...,...,...,...,...,...,...,...,...,...
995,495,2004-09-07-discussion-sedalia-missouri,"The third lesson is, is that we must deal with...",rep,2004,False,NaN,True,120.0,2004-09-07-discussion-sedalia-missouri_120.0
996,496,3171,"So, here we go. Governor Clinton--I honestly b...",rep,1992,False,NaN,True,3.0,3171_3.0
997,497,1829,"This is not the time to go into it in detail, ...",rep,1960,True,NaN,True,6.0,1829_6.0
998,498,2090,My only prayer is that I will have the strengt...,dem,1968,False,NaN,True,14.0,2090_14.0


In [9]:
unsampled_dt = all_dt[~all_dt['total_id'].isin(sample['total_id'])]
unsampled_dt

,Unnamed: 0,Speech_id,text,party,term,comp,populist_old_keywords,par_id,total_id
0,0,2682,"When we start talking about the economy, it's ...",rep,1980,False,True,1,2682_1
1,1,2682,It's no secret which groups are hit the hardes...,rep,1980,False,False,2,2682_2
2,2,2682,How can our elderly who have worked so hard to...,rep,1980,True,False,3,2682_3
3,3,2682,I believe that social security is one of this ...,rep,1980,True,True,4,2682_4
4,4,2682,"In contrast, I am committed to an economic pro...",rep,1980,True,False,5,2682_5
...,...,...,...,...,...,...,...,...,...
71822,72341,2000-08-17-national-convention-los,"And I ask all of you, my fellow citizens, from...",dem,2000,True,False,84,2000-08-17-national-convention-los_84
71823,72342,2000-08-17-national-convention-los,"I know my own imperfections. For example, I kn...",dem,2000,False,False,85,2000-08-17-national-convention-los_85
71824,72344,2000-08-17-national-convention-los,Vice President Gore. But the presidency is mor...,dem,2000,True,False,87,2000-08-17-national-convention-los_87
71825,72345,2000-08-17-national-convention-los,There are big choices ahead and our whole futu...,dem,2000,True,False,88,2000-08-17-national-convention-los_88


In [15]:
# drop paragraphs already sampled
new_dat = (pd.merge(all_dt, sample[['Speech_id', 'text', 'party', 'term', 'comp']], indicator=True, how='outer')
         .query('_merge=="left_only"')
         .drop('_merge', axis=1))

new_dat

,Unnamed: 0,Speech_id,text,party,term,comp,populist_old_keywords,par_id
0,0.0,2682,"When we start talking about the economy, it's ...",rep,1980,False,True,1.0
1,1.0,2682,It's no secret which groups are hit the hardes...,rep,1980,False,False,2.0
2,2.0,2682,How can our elderly who have worked so hard to...,rep,1980,True,False,3.0
3,3.0,2682,I believe that social security is one of this ...,rep,1980,True,True,4.0
4,4.0,2682,"In contrast, I am committed to an economic pro...",rep,1980,True,False,5.0
...,...,...,...,...,...,...,...,...
71857,72341.0,2000-08-17-national-convention-los,"And I ask all of you, my fellow citizens, from...",dem,2000,True,False,84.0
71858,72342.0,2000-08-17-national-convention-los,"I know my own imperfections. For example, I kn...",dem,2000,False,False,85.0
71859,72344.0,2000-08-17-national-convention-los,Vice President Gore. But the presidency is mor...,dem,2000,True,False,87.0
71860,72345.0,2000-08-17-national-convention-los,There are big choices ahead and our whole futu...,dem,2000,True,False,88.0


In [120]:
# merge and clean up new dat 
all_dt = pd.read_csv('20201110_all_paragraphs.csv')
merged = new_dat.merge(all_dt, how = "left", on = "text")
merged = merged[merged['Speech_id_x'] == merged['Speech_id_y']] 
del merged['Speech_id_y']

merged = merged.drop_duplicates(subset = ["Speech_id_x", "par_id"])


del merged["Unnamed: 0_x"]
del merged["Unnamed: 0_y"]
del merged["party_y"]
del merged["term_y"]
del merged["comp_y"]

pd.options.display.max_colwidth = 200
merged = merged[merged['text'].notna()] #remove na 
audience = merged[merged['text'].str.contains('Audience')] # check where the audience reaction is 
audience = audience[audience['text'].str.split().str.len()<13] # most audience reactions are short 
merged = (pd.merge(merged, audience, indicator=True, how='outer')
         .query('_merge=="left_only"')
         .drop('_merge', axis=1))

merged.rename(columns = {'Speech_id_x':'Speech_id', 'party_x':"party", "term_x":"term", "comp_x":"comp", "populist_old_keywords_x":"populist_old_keywords"}, inplace = True) 
del merged['populist_old_keywords_y']

In [16]:
# SAMPLE
import random
import pandas as pd
# merged = pd.read_csv('20201115_all_paragraphs.csv')
random.seed(11)
new_sample = new_dat.sample(n = 250)

# # first 500 completely random
# sample1 = sample[0:500]
# # second sample = 250 random + 250 populist keyword == T
# sample2_1 = sample[500:750]

# sample = sample1.append(sample2_1, ignore_index = True)

# # exclude the ones already sampled 
# new_sample = (pd.merge(merged, sample, indicator=True, how='outer')
#          .query('_merge=="left_only"')
#          .drop('_merge', axis=1))
# sample2_2 = new_sample[new_sample['populist_old_keywords'] == True].sample(n = 250)

# sample2 = sample2_1.append(sample2_2, ignore_index = True)

In [24]:
new_sample['par_id'] = new_sample['par_id'].astype(int)
new_sample['total_id'] = new_sample['Speech_id'].astype(str)  + '_' + new_sample['par_id'].astype(str) 
new_sample = new_sample[["total_id",'Speech_id',"par_id", "text", "party", "term", "comp", "populist_old_keywords"]]
#sample1.to_csv('bart_2nd_sample.csv')
#sample2.to_csv('yuchen_2nd_sample.csv')
new_sample.to_csv('oscar_2nd_sample.csv')

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [107]:
merged[merged['text'].str.contains('Audience')]

,Speech_id_x,text,party_x,term_x,comp_x,par_id
198,2133,"Mr. Nixon has come to this city and he has brought a message on the date of October 23rd which says the ""U. S. World Lead Lost, Nixon Tells Battle Creek Audience,"" and what did he say? He said we...",dem,1968,False,26
15470,2804,"Well now, it's time for me to go-- [Audience: No!] Yes--oh--have to. But I see so many families here this afternoon, so many fine young people. May I just leave you with one last thought from my h...",rep,1984,True,26
50512,2012-09-27-virginia-beach-virginia,The President. Jim Webb! Audience members. Four more years! Four more years! Four more years!,dem,2012,True,0
50518,2012-09-27-virginia-beach-virginia,"Audience members. Yours! The President. No, well, I know some in the crowd may be a little biased, but—[Laughter]—but I also want to speak to the audience who may be seeing this over the television.",dem,2012,True,7
50528,2012-09-27-virginia-beach-virginia,"Number one, I want to export more products and I want to outsource fewer jobs. When my opponent said we should just ""let Detroit go bankrupt""—— Audience members. Boo!",dem,2012,True,18
...,...,...,...,...,...,...
69728,2016-10-12-pavilion-ocala-florida,"And who is going to pay for the wall? Audience: ""Mexico"" 100 percent. They just don't know it yet. They don't know it yet, but 100 percent. They'll pay for the wall.",rep,2016,True,116
71297,2000-08-17-national-convention-los,"You know... Audience. Go, Al, go. Go, Al, go. Go, Al, go. Go, Al, go. Go, Al, go.",dem,2000,True,31
71298,2000-08-17-national-convention-los,"Vice President Gore. Are you with me? Audience. Go, Al, go. Go, Al, go. Go, Al, go.",dem,2000,True,32
71312,2000-08-17-national-convention-los,"And all of this...[applause] Audience. Go, Al, go. Go, Al, go. Go, Al, go. Go, Al, go. Go, Al, go.",dem,2000,True,48


In [136]:
merged.to_csv('20201115_all_paragraphs.csv')

# Add paragraph id

In [20]:
# 
bart_sample = pd.read_csv("bart_sample.csv")
bart_annot = pd.read_csv("annotated_par_populism_bart.csv")
all_dat = pd.read_csv("20201115_all_paragraphs.csv")

#print(bart_sample)
print(bart_annot)
#print(all_dat)

#all_dat[0:50]

     Unnamed: 0                                               text  \
0             0  The future is my responsibility and it's yours...   
1             1  I ask you to help us make new history and I as...   
2             2  But, getting back to the issues before us, I s...   
3             3  It used to be in America all the racial issues...   
4             4  Mr. Lamb. Yes, it's a legal term. The Presiden...   
..          ...                                                ...   
492         766  Ten months ago, in a tragic moment, I was call...   
493         767  He says he cares about the middle class, but h...   
494         768  We have got to have a tough plan to bring heal...   
495         769  The constant partisan rancor that stops us fro...   
496         770  We began working on an energy policy, which th...   

            label   annotated_at                               Speech_id_x  \
0    not populist  11/4/20 15:28                                      2600   
1  

In [23]:
bart_annot = bart_annot[['Speech_id_x', 'par_id_y', 'text', 'party_x','term_x','comp_x','label']]

cols = ['Speech_id','par_id','text', 'party','term','comp','label']
 
bart_annot.columns = cols



KeyError: "['Speech_id_x', 'comp_x', 'party_x', 'term_x', 'par_id_y'] not in index"

In [24]:
print(bart_annot)
bart_annot.to_csv("bart_annotated.csv")

                                    Speech_id  par_id  \
0                                        2600     8.0   
1                                        2005    27.0   
2                                        1871     3.0   
3                                        3256     9.0   
4      2004-09-07-discussion-sedalia-missouri    47.0   
..                                        ...     ...   
492                                      1922    21.0   
493                                      2868    14.0   
494                                      3004    31.0   
495  2008-10-01-remarks-independence-missouri    16.0   
496                                      2575    11.0   

                                                  text party  term   comp  \
0    The future is my responsibility and it's yours...   dem  1980  False   
1    I ask you to help us make new history and I as...   dem  1968   True   
2    But, getting back to the issues before us, I s...   rep  1960   True   
3    It

In [13]:
#### add par_id to Yuchen first sample


oscar_sample = pd.read_csv('oscar_sample.csv')


oscar_sample['drawn'] = ['randomly'] *250 + ['keyword'] * 250

oscar_sample

,Unnamed: 0,Speech_id,text,party,term,comp,populist_old_keywords,par_id,drawn
0,0,2162,But don't worry about that. You know who is in...,rep,1968,False,False,3,randomly
1,1,2016-10-28-manchester-new-hampshire,And these people are seriously corrupt. Terrib...,rep,2016,True,True,21,randomly
2,2,1045,We can make our society ever more productive a...,dem,1952,True,False,13,randomly
3,3,2397,Q. Do you think that Gerald Ford had anything ...,dem,1976,True,False,30,randomly
4,4,2012-10-05-rally-fairfax-virginia,One of the main reasons record surpluses under...,dem,2012,False,False,9,randomly
...,...,...,...,...,...,...,...,...,...
495,495,2004-09-07-discussion-sedalia-missouri,"The third lesson is, is that we must deal with...",rep,2004,False,True,120,keyword
496,496,3171,"So, here we go. Governor Clinton--I honestly b...",rep,1992,False,True,3,keyword
497,497,1829,"This is not the time to go into it in detail, ...",rep,1960,True,True,6,keyword
498,498,2090,My only prayer is that I will have the strengt...,dem,1968,False,True,14,keyword


In [10]:
yuchen_sample = yuchen_annot.drop('label', axis = 1)


,Unnamed: 0,Speech_id,par_id,text,party,term,comp,drawn
0,0,2192,27,The fact of the matter is that we find in the ...,rep,1968,True,randomly
1,1,1987,25,The first thing you do Tuesday morning is get ...,dem,1964,False,randomly
2,2,3156,11,"Governor Clinton is talking about ""Well, we re...",rep,1992,False,randomly
3,3,1402,18,"In the meantime, wages have been raised and th...",rep,1956,False,randomly
4,4,3226,41,"And when people need it, they're smart enough ...",dem,1996,False,randomly
...,...,...,...,...,...,...,...,...
490,490,2004-10-26-dubuque-iowa-0,34,You cannot be pro-doctor and pro-patient and p...,rep,2004,False,randomly
491,491,1076,4,"We are not only, as I say, a moral countr...",dem,1952,False,randomly
492,492,1659,8,The housing--if there is any state in the unio...,dem,1960,False,randomly
493,493,2008-11-03-jacksonville-florida-1,31,Don't believe for a second this election is ov...,dem,2008,False,randomly


# Remove All Caps Tags

In [182]:
annenberg = pd.read_csv("paragraphs_dropped_annenberg.csv")

In [129]:
for i in range(len(annenberg['text'])):
    s = annenberg['text'][i]
    s = s.translate(str.maketrans('', '', string.punctuation)) #remove punctuations
    all_caps = "".join(re.findall(r"[A-Z]{3,}.*$", s))# extract caps (start from the end, greedy search for all caps of 3 or more) 
    annenberg['text'][i] = re.sub(all_caps, '', s)
    

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [184]:
for i in range(len(annenberg['text'])):
    try:
        annenberg['text'][i] = re.match(r'^.+?(?=[A-Z]{3,})', annenberg['text'][i]).group()
    except:
        continue

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [193]:
annenberg["WordsCount"]=annenberg["text"].apply(lambda x: len(x))
annenberg = annenberg[annenberg["WordsCount"]> 2]

In [ ]:
for speech_id in annenberg['Speech_id']:
    annenberg[speech_id]['par_id'] = np.where(len(annenberg[speech_id])>1, 

In [194]:
annenberg.to_csv('annenberg_dropped_pars_cleaned.csv')

In [199]:
annenberg['par_id'] = 998
annenberg[annenberg.duplicated('Speech_id'), keep = "last"]

SyntaxError: invalid syntax (<ipython-input-199-bddc64ae40fe>, line 2)

In [196]:
annenberg

,text,Speech_id,party,term,comp,WordsCount,par_id
0,"But we can turn those policies around, reinvig...",2682,rep,1980,True,177,998
1,Only you can put peace and prosperity within o...,2696,rep,1980,True,134,998
4,A democracy is a live society--and growth is t...,906,dem,1952,False,503,998
5,You will hear a lot about the need for a chang...,906,dem,1952,False,216,998
6,Thank you all very much.,2872,rep,1984,True,25,998
...,...,...,...,...,...,...,...
3342,"Just as I came here in 1966, I come here again...",2658,rep,1980,True,111,998
3343,"Thank you, good night, and God bless you.",2880,rep,1984,True,42,998
3344,We've got to stop this giveaway of our natural...,1386,dem,1956,True,369,998
3346,And now I'll just finish by saying something t...,2857,rep,1984,True,192,998
